In [1]:
from datasets import load_dataset

ds = load_dataset("JacobLinCool/VoiceBank-DEMAND-16k", use_auth_token=True, cache_dir="/storage/hdd0/data/mayank_dataset")

/home/mayank.mds2023/miniconda3/envs/aml/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration JacobLinCool--VoiceBank-DEMAND-16k-2c58d2d1c1a56ab1
Reusing dataset parquet (/storage/hdd0/data/mayank_dataset/JacobLinCool___parquet/JacobLinCool--VoiceBank-DEMAND-16k-2c58d2d1c1a56ab1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 618.77it/s]


In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64'

import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

1.4.0
10.1
True


In [3]:
ds.keys()

dict_keys(['test', 'train'])

In [4]:
print(ds)

DatasetDict({
    test: Dataset({
        features: ['id', 'clean', 'noisy'],
        num_rows: 824
    })
    train: Dataset({
        features: ['id', 'clean', 'noisy'],
        num_rows: 11572
    })
})


In [5]:
ds['test'][0]

{'id': 'p232_001',
 'clean': {'path': 'p232_001.wav',
  'array': array([ 0.00213623,  0.00323486,  0.00244141, ..., -0.00576782,
         -0.00543213, -0.00354004]),
  'sampling_rate': 16000},
 'noisy': {'path': 'p232_001.wav',
  'array': array([ 0.01589966,  0.02572632,  0.02059937, ..., -0.00421143,
         -0.00469971, -0.00354004]),
  'sampling_rate': 16000}}

In [8]:
for example in ds['train']:
    print(example)
    break

{'id': 'p226_001', 'clean': {'path': 'p226_001.wav', 'array': array([-0.00286865, -0.00497437, -0.00469971, ..., -0.00201416,
       -0.00152588, -0.00021362]), 'sampling_rate': 16000}, 'noisy': {'path': 'p226_001.wav', 'array': array([-0.00283813, -0.00488281, -0.00473022, ...,  0.006073  ,
        0.00564575,  0.00540161]), 'sampling_rate': 16000}}


In [9]:
example.keys()

dict_keys(['id', 'clean', 'noisy'])

In [10]:
clean = example['clean']
clean.keys()

dict_keys(['path', 'array', 'sampling_rate'])

In [11]:
clean['array'].shape

(36480,)

In [12]:
example['noisy']['array'].shape

(36480,)

In [14]:
type(ds['train'])

datasets.arrow_dataset.Dataset

In [22]:
ds1 = ds['train'].train_test_split(test_size=0.1, seed=42)

In [23]:
ds1.keys()

dict_keys(['train', 'test'])

In [24]:
dataset_dict = {"train": ds1['train'], "val": ds1['test'], "test": ds['test']}

In [25]:
ds1['test'].shape

(1158, 3)

In [26]:
ds['test'].shape

(824, 3)

In [4]:
temp = ds['train'].train_test_split(test_size=0.1, seed=42)
dataset = {
    "train": temp['train'],
    "val": temp['test'],
    "test": ds['test']
}

Loading cached split indices for dataset at /storage/hdd0/data/mayank_dataset/JacobLinCool___parquet/JacobLinCool--VoiceBank-DEMAND-16k-2c58d2d1c1a56ab1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-4ea28409c9bd1193.arrow and /storage/hdd0/data/mayank_dataset/JacobLinCool___parquet/JacobLinCool--VoiceBank-DEMAND-16k-2c58d2d1c1a56ab1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-cbb577a008396ea1.arrow


In [5]:
dataset['test']

Dataset({
    features: ['id', 'clean', 'noisy'],
    num_rows: 824
})

In [6]:
for example in dataset['test']:
    print(example)
    break

{'id': 'p232_001', 'clean': {'path': 'p232_001.wav', 'array': array([ 0.00213623,  0.00323486,  0.00244141, ..., -0.00576782,
       -0.00543213, -0.00354004]), 'sampling_rate': 16000}, 'noisy': {'path': 'p232_001.wav', 'array': array([ 0.01589966,  0.02572632,  0.02059937, ..., -0.00421143,
       -0.00469971, -0.00354004]), 'sampling_rate': 16000}}


In [8]:
example['noisy'].keys()

dict_keys(['path', 'array', 'sampling_rate'])

In [10]:
example['noisy']['path']

'p232_001.wav'

In [ ]:
# read pkl file
import pickle


with open('./checkpoints/waveunet/results.pkl', 'rb') as f:
    test_data = pickle.load(f)
len(test_data)

824

In [2]:
test_data[0]

{'clean': {'SDR': array([23.12008813]),
  'ISR': array([30.36611248]),
  'SIR': array([inf]),
  'SAR': array([23.96229591])}}

In [1]:
from tensorboard.backend.event_processing import event_accumulator

# Load the event file
ea = event_accumulator.EventAccumulator("./logs/waveunet/events.out.tfevents.1743997275.gpuserver.613521.0")
ea.Reload()

# List all tags
print("Tags:", ea.Tags())

# # Read scalar values
# scalars = ea.Scalars("loss")
# for scalar in scalars:
#     print(f"Step: {scalar.step}, Value: {scalar.value}")


Tags: {'images': [], 'audio': ['input', 'clean_pred', 'clean_target'], 'histograms': [], 'scalars': ['lr', 'train_loss', 'val_loss', 'test_loss', 'test_SDR_clean', 'test_SIR_clean', 'test_SDR'], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}


In [4]:
ea.Scalars("train_loss")

[ScalarEvent(wall_time=1743997337.5142257, step=17, value=0.021342266350984573),
 ScalarEvent(wall_time=1743997338.0386367, step=24, value=0.0322040356695652),
 ScalarEvent(wall_time=1743997340.5164647, step=57, value=0.023286817595362663),
 ScalarEvent(wall_time=1743997341.2657413, step=67, value=0.02700246311724186),
 ScalarEvent(wall_time=1743997341.7139957, step=73, value=0.022533664479851723),
 ScalarEvent(wall_time=1743997342.378254, step=82, value=0.020497137680649757),
 ScalarEvent(wall_time=1743997347.018965, step=143, value=0.02402653731405735),
 ScalarEvent(wall_time=1743997347.2494702, step=146, value=0.017010003328323364),
 ScalarEvent(wall_time=1743997347.6936293, step=152, value=0.015613837167620659),
 ScalarEvent(wall_time=1743997349.472311, step=176, value=0.017231658101081848),
 ScalarEvent(wall_time=1743997350.4323163, step=189, value=0.022039642557501793),
 ScalarEvent(wall_time=1743997353.808736, step=228, value=0.016987863928079605),
 ScalarEvent(wall_time=1743997